In [2]:
from utils.v3 import yolov3
import torch
import numpy as np
from torchsummary import summary
import yaml
import os

In [50]:
path = "./yodata/data.yml"
with open(path, "r") as f:
    data = yaml.load(f, Loader=yaml.SafeLoader)

data

{'names': ['car'],
 'nc': 1,
 'path': './yodata',
 'test': 'test',
 'train': 'train',
 'val': 'val'}

In [55]:
data["path"] + "/" + data["train"]

'./yodata/train'

In [39]:
boxes = []
with open(os.path.join(path, "vid_4_700" + ".txt"), "r") as f:
    for line in f:
        boxes.append(line.split("\n")[0].split(" "))
boxes

[['0',
  '0.9337916063609467',
  '0.564913127368421',
  '0.13241678727810652',
  '0.17631917631578944'],
 ['0',
  '0.562590448668639',
  '0.6324806949999999',
  '0.38422575976331363',
  '0.2265122263157895']]

In [41]:
bboxes = np.roll(np.loadtxt(fname=os.path.join(path, "vid_4_700"+".txt"), delimiter=" "), shift=4, axis=0).tolist()

In [42]:
bboxes

[[0.0,
  0.9337916063609467,
  0.564913127368421,
  0.13241678727810652,
  0.17631917631578944],
 [0.0,
  0.562590448668639,
  0.6324806949999999,
  0.38422575976331363,
  0.2265122263157895]]

In [8]:
image = torch.randint(0, 255, (2,3,13,13,85)).float()

In [13]:
grid_len = np.arange(13)
a,b = np.meshgrid(grid_len,grid_len)

x_offset = torch.FloatTensor(a).view(-1,1)
y_offset = torch.FloatTensor(b).view(-1,1)

x_y_offset = torch.cat((x_offset, y_offset), 1).repeat(1,9).view(-1,2).unsqueeze(0)

print(x_offset.shape)


torch.Size([169, 1])


In [11]:
x_y_offset.shape

torch.Size([1, 1521, 2])

In [36]:
ANCHORS = [ 
    [(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)], 
    [(0.07, 0.15), (0.15, 0.11), (0.14, 0.29)], 
    [(0.02, 0.03), (0.04, 0.07), (0.08, 0.06)], 
] 
anchors = torch.Tensor(ANCHORS[0] + ANCHORS[1] + ANCHORS[2])
num_per_scale = anchors.shape[0] // 3
print(num_per_scale)
torch.tensor(ANCHORS).shape

3


torch.Size([3, 3, 2])

In [24]:
boxes = torch.ran(2,5)
boxes

tensor([[-1.2919,  1.0112,  0.2272,  1.0376,  1.6589],
        [-1.5730,  0.9015,  0.4778,  0.9910, -0.8286]])

In [25]:
def iou(box1, box2, is_pred=True): 
    if is_pred: 
        # IoU score for prediction and label 
        # box1 (prediction) and box2 (label) are both in [x, y, width, height] format 
          
        # Box coordinates of prediction 
        b1_x1 = box1[..., 0:1] - box1[..., 2:3] / 2
        b1_y1 = box1[..., 1:2] - box1[..., 3:4] / 2
        b1_x2 = box1[..., 0:1] + box1[..., 2:3] / 2
        b1_y2 = box1[..., 1:2] + box1[..., 3:4] / 2
  
        # Box coordinates of ground truth 
        b2_x1 = box2[..., 0:1] - box2[..., 2:3] / 2
        b2_y1 = box2[..., 1:2] - box2[..., 3:4] / 2
        b2_x2 = box2[..., 0:1] + box2[..., 2:3] / 2
        b2_y2 = box2[..., 1:2] + box2[..., 3:4] / 2
  
        # Get the coordinates of the intersection rectangle 
        x1 = torch.max(b1_x1, b2_x1) 
        y1 = torch.max(b1_y1, b2_y1) 
        x2 = torch.min(b1_x2, b2_x2) 
        y2 = torch.min(b1_y2, b2_y2) 
        # Make sure the intersection is at least 0 
        intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0) 
  
        # Calculate the union area 
        box1_area = abs((b1_x2 - b1_x1) * (b1_y2 - b1_y1)) 
        box2_area = abs((b2_x2 - b2_x1) * (b2_y2 - b2_y1)) 
        union = box1_area + box2_area - intersection 
  
        # Calculate the IoU score 
        epsilon = 1e-6
        iou_score = intersection / (union + epsilon) 
  
        # Return IoU score 
        return iou_score 
      
    else: 
        # IoU score based on width and height of bounding boxes 
          
        # Calculate intersection area 
        intersection_area = torch.min(box1[..., 0], box2[..., 0]) * torch.min(box1[..., 1], box2[..., 1]) 
  
        # Calculate union area 
        box1_area = box1[..., 0] * box1[..., 1] 
        box2_area = box2[..., 0] * box2[..., 1] 
        union_area = box1_area + box2_area - intersection_area 
  
        # Calculate IoU score 
        iou_score = intersection_area / union_area 
  
        # Return IoU score 
        return iou_score

In [28]:
grid = [13, 26, 52]

target = [torch.zeros((3, s, s, 6)) for s in grid]
target[0].shape

torch.Size([3, 13, 13, 6])

In [44]:
anchor = [0,1,2,3,4,5,6,7,8]
for i in anchor:
    print(i // 3)

0
0
0
1
1
1
2
2
2


In [43]:
anchor = [0,1,2,3,4,5,6,7,8]
for i in anchor:
    print(i % 3)

0
1
2
0
1
2
0
1
2


In [31]:
for box in boxes:
    iou_anchors = iou(torch.Tensor(box[2:4]), anchors, is_pred=False)
    anchor_indices = iou_anchors.argsort(descending=True, dim=0)
    print(anchor_indices)
    x, y, h, w, label = box
    has_anchor = [False] * 3
    for anchor_idx in anchor_indices: 
        scale_idx = anchor_idx // 3 
        print(scale_idx)
        anchor_on_scale = anchor_idx % 3 
        print(anchor_on_scale)

        s = grid[scale_idx]
        print("s:", s)
        i, j = int(s * y), int(s * x) 
        print((i,j))
        anchor_taken = target[scale_idx][anchor_on_scale, i, j, 0]
        print(anchor_taken)

        if not anchor_taken and not has_anchor[scale_idx]: 
  
                    # Set the probability to 1 
                target[scale_idx][anchor_on_scale, i, j, 0] = 1
  
                    # Calculating the center of the bounding box relative 
                    # to the cell 
                x_cell, y_cell = s * x - j, s * y - i  
  
                    # Calculating the width and height of the bounding box  
                    # relative to the cell 
                width_cell, height_cell = (w * s, h * s) 
  
                    # Idnetify the box coordinates 
                box_coordinates = torch.tensor( 
                                        [x_cell, y_cell, width_cell,  
                                         height_cell] 
                                    ) 
  
                    # Assigning the box coordinates to the target 
                target[scale_idx][anchor_on_scale, i, j, 1:5] = box_coordinates 
  
                    # Assigning the class label to the target 
                target[scale_idx][anchor_on_scale, i, j, 5] = int(class_label) 
  
                    # Set the anchor box as assigned for the scale 
                has_anchor[scale_idx] = True

tensor([1, 2, 0, 5, 4, 3, 8, 7, 6])
tensor(0)
tensor(1)
s: 13
(13, -16)


IndexError: index 13 is out of bounds for dimension 1 with size 13

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = yolov3(n_classes=80).to(device)
print(model)

yolov3(
  (backbone): darknet(
    (conv): CNNBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (conv1): CNNBlock(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (res1): Residual_Block(
      (blocks): ModuleList(
        (0): Block(
          (conv_1): CNNBlock(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): LeakyReLU(negative_slope=0.01, inplace=True)
          )
          (conv_2): CNNBlock(
            (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1

In [ ]:
summary(model, (3,416,416))